In [1]:
import numpy as np
import numba as nb
import os
import cv2
import pickle
import time
from math import floor
from Functions import *
from multiprocessing import Pool
import numba as nb
import warnings

/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
trainPath = 'trainingData'

R = 2                           # Upscaling factor=2
patchSize = 11                  # Pacth Size=11
gradientSize = 9                # Gradient Size = 9
Qangle = 24                     # Quantization factor of angle =24
Qstrength = 3                   # Quantization factor of strength =3
Qcoherence = 3                  # Quantization factor of coherence =3
stre = np.zeros((Qstrength-1))  # Strength boundary
cohe = np.zeros((Qcoherence-1)) # Coherence boundary

Q = np.zeros((R*R, Qangle*Qstrength*Qcoherence, patchSize*patchSize, patchSize*patchSize))  # Eq.4
V = np.zeros((R*R, Qangle*Qstrength*Qcoherence, patchSize*patchSize))                       # Eq.5
h = np.zeros((R*R, Qangle*Qstrength*Qcoherence, patchSize*patchSize))
mark = np.zeros((R*R, Qangle*Qstrength*Qcoherence))                  # statical distribution of patch numbers in each bucket
w = Gaussian2d([patchSize, patchSize], 2)
w = w/w.max()
w = np.diag(w.ravel())                                               # Diagnal weighting matrix Wk in Algorithm 1

filelist = make_dataset(trainPath)

instance = 20000000                          # use 20000000 patches to get the Strength and coherence boundary

patchNumber = 0                              # patch number has been used
quantization = np.zeros((instance,2))        # quantization boundary

In [3]:
for image in filelist:
    print('\r', end='')
    print('' * 60, end='')
    print('\r Quantization: Processing '+ str(instance/2) + ' patches (' + str(200*patchNumber/instance) + '%)')
    im_uint8 = cv2.imread(image)
    if is_greyimage(im_uint8):
        im_uint8 = im_uint8[:,:,0]
    if len(im_uint8.shape)>2:
        im_ycbcr = BGR2YCbCr(im_uint8)
        im = im_ycbcr[:,:,0]
    else:
        im = im_uint8
    im = modcrop(im,R)
    im_LR = Prepare(im,patchSize,R)         # Prepare the cheap-upscaling images (optional: JPEG compression)
    im_GX,im_GY = np.gradient(im_LR)        # Calculate the gradient images
    quantization, patchNumber = QuantizationProcess (im_GX, im_GY,patchSize, patchNumber, w, quantization)  # get the strength and coherence of each patch
    if (patchNumber > instance/2):
        break
print('\r done')

 Quantization: Processing 10000000.0 patches (0.0%)


/usr/local/anaconda3/lib/python3.7/site-packages/numba/compiler.py:588: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see http://numba.pydata.org/numba-doc/latest/user/parallel.html#diagnostics for help.

File "Functions.py", line 130:
@nb.jit(nopython=True, parallel=True)
def Grad(patchX,patchY,weight):
^

  self.func_ir.loc))


 Quantization: Processing 10000000.0 patches (1.4617%)
 Quantization: Processing 10000000.0 patches (2.9234%)
 Quantization: Processing 10000000.0 patches (4.3851%)
 Quantization: Processing 10000000.0 patches (5.8468%)
 Quantization: Processing 10000000.0 patches (7.3085%)
 Quantization: Processing 10000000.0 patches (8.7702%)
 Quantization: Processing 10000000.0 patches (10.2319%)
 Quantization: Processing 10000000.0 patches (11.6936%)
 Quantization: Processing 10000000.0 patches (13.1553%)
 Quantization: Processing 10000000.0 patches (14.617%)
 Quantization: Processing 10000000.0 patches (16.0787%)
 Quantization: Processing 10000000.0 patches (17.5404%)
 Quantization: Processing 10000000.0 patches (19.0021%)
 Quantization: Processing 10000000.0 patches (20.4638%)
 Quantization: Processing 10000000.0 patches (21.9255%)
 Quantization: Processing 10000000.0 patches (23.3872%)
 Quantization: Processing 10000000.0 patches (24.8489%)
 Quantization: Processing 10000000.0 patches (26.3106%)

In [4]:
# uniform quantization of patches, get the optimized strength and coherence boundaries
quantization = quantization [0:patchNumber,:]
quantization = np.sort(quantization,axis=0)
for i in range(Qstrength-1):
    stre[i] = quantization[floor((i+1)*patchNumber/Qstrength),0]
for i in range(Qcoherence-1):
    cohe[i] = quantization[floor((i+1)*patchNumber/Qcoherence),1]

In [5]:
print('Begin to process images:')
start = time.time()
imagecount = 1
for image in filelist:
    print('\r', end='')
    print('' * 60, end='')
    print('\r Processing ' + str(imagecount) +'/' + str(len(filelist))+ ' image ('   + image + ')')
    
    im_uint8 = cv2.imread(image)
    if is_greyimage(im_uint8):
        im_uint8 = im_uint8[:,:,0]
    if len(im_uint8.shape)>2:
        im_ycbcr = BGR2YCbCr(im_uint8)
        im = im_ycbcr[:,:,0]
    else:
        im = im_uint8
    im = modcrop(im,R)
    im_LR = Prepare(im,patchSize,R)
    im_HR = im2double(im)
    #im_HR = Dog1(im_HR)                # optional: sharpen the image
    im_GX,im_GY = np.gradient(im_LR)
    Q, V, mark = TrainProcessSMC(im_LR, im_HR, im_GX, im_GY,patchSize, w, Qangle, Qstrength,Qcoherence, stre, cohe, R, Q, V, mark)  # get Q, V of each patch
    imagecount += 1
end = time.time()
print(end - start)

Begin to process images:
 Processing 1/200 image (trainingData/157036.jpg)


/usr/local/anaconda3/lib/python3.7/site-packages/numba/compiler.py:588: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see http://numba.pydata.org/numba-doc/latest/user/parallel.html#diagnostics for help.

File "Functions.py", line 145:
@nb.jit(nopython=True, parallel=True)
def HashTable(patchX,patchY,weight, Qangle,Qstrength,Qcoherence,stre,cohe):
^

  self.func_ir.loc))


 Processing 2/200 image (trainingData/187029.jpg)
 Processing 3/200 image (trainingData/71046.jpg)
 Processing 4/200 image (trainingData/299091.jpg)
 Processing 5/200 image (trainingData/242078.jpg)
 Processing 6/200 image (trainingData/232038.jpg)
 Processing 7/200 image (trainingData/173036.jpg)
 Processing 8/200 image (trainingData/198054.jpg)
 Processing 9/200 image (trainingData/388016.jpg)
 Processing 10/200 image (trainingData/292066.jpg)
 Processing 11/200 image (trainingData/189011.jpg)
 Processing 12/200 image (trainingData/59078.jpg)
 Processing 13/200 image (trainingData/108041.jpg)
 Processing 14/200 image (trainingData/66075.jpg)
 Processing 15/200 image (trainingData/20008.jpg)
 Processing 16/200 image (trainingData/370036.jpg)
 Processing 17/200 image (trainingData/41025.jpg)
 Processing 18/200 image (trainingData/207056.jpg)
 Processing 19/200 image (trainingData/314016.jpg)
 Processing 20/200 image (trainingData/188091.jpg)
 Processing 21/200 image (trainingData/56028

In [6]:
# optional: Using patch symmetry for nearly 8* more learning examples
print('\r', end='')
print(' ' * 60, end='')
print('\r Using patch symmetry for nearly 8* more learning examples:')
for i in range(Qangle):
    for j in range(Qstrength*Qcoherence):
        for r in range(R*R):
            for t in range(1,8):
                t1 = t % 4
                t2 = floor(t / 4)
                Q1 = Getfromsymmetry1(Q[r, i * Qstrength * Qcoherence + j], patchSize, t1, t2)  # Rotate 90*t1 degree or flip t2
                V1 = Getfromsymmetry2(V[r, i * Qstrength * Qcoherence + j], patchSize, t1, t2)
                i1 = Qangle*t1/2 + i
                i1 = np.int(i1)
                if t2 == 1:
                    i1 = Qangle -1 - i1
                while i1 >= Qangle:
                    i1 = i1 - Qangle
                while i1 < 0:
                    i1 = i1 + Qangle
                Q[r, i1 * Qstrength * Qcoherence + j] += Q1
                V[r, i1 * Qstrength * Qcoherence + j] += V1

 Using patch symmetry for nearly 8* more learning examples: 


# Concurrent trainning mode

In [5]:
def ConcurenrtTrainprocessSMC(i):
    if (i < iteration - 1):
        offset_i = offset[i * batch:i * batch + batch]
        offset2_i = offset2[i * batch:i * batch + batch]
        grid = np.tile(gridon[..., None], [1, 1, batch]) + np.tile(offset_i, [patchSize, patchSize, 1])
    else:
        offset_i = offset[i * batch:im.size]
        offset2_i = offset2[i * batch:im.size]
        grid = np.tile(gridon[..., None], [1, 1, im.size - (iteration - 1) * batch]) + np.tile(offset_i,[patchSize, patchSize,1])
    f = im_LR.ravel()[grid]
    gx = im_GX.ravel()[grid]
    gy = im_GY.ravel()[grid]
    gx = gx.reshape((1, patchSize * patchSize, gx.shape[2]))
    gy = gy.reshape((1, patchSize * patchSize, gy.shape[2]))
    G = np.vstack((gx, gy))
    g1 = np.transpose(G, (2, 0, 1))
    g2 = np.transpose(G, (2, 1, 0))
    x = Gaussian_Mul(g1, g2, wGaussian)
    w, v = np.linalg.eig(x)
    idx = (-w).argsort()
    w = w[np.arange(np.shape(w)[0])[:, np.newaxis], idx]
    v = v[np.arange(np.shape(v)[0])[:, np.newaxis, np.newaxis], np.arange(np.shape(v)[1])[np.newaxis, :, np.newaxis], idx[:,np.newaxis,:]]
    thelta = np.arctan(v[:, 1, 0] / v[:, 0, 0])
    thelta[thelta < 0] = thelta[thelta < 0] + pi
    thelta = np.floor(thelta / (pi / Qangle))
    thelta[thelta > Qangle - 1] = Qangle - 1
    thelta[thelta < 0] = 0
    lamda = w[:, 0]
    u = (np.sqrt(w[:, 0]) - np.sqrt(w[:, 1])) / (np.sqrt(w[:, 0]) + np.sqrt(w[:, 1]) + 0.00000000000000001)
    lamda = np.searchsorted(stre, lamda)
    u = np.searchsorted(cohe, u)
    j = thelta * Qstrength * Qcoherence + lamda * Qcoherence + u
    j = j.astype('int')
    
    f_vec = np.reshape(f, (patchSize * patchSize, -1)) - mean
    f_vec_reduced = np.dot(pca_matrix, f_vec)
    t = kmeans.predict(f_vec_reduced.reshape(-1, 1))
    
    f = f.reshape(-1, patchSize*patchSize)
    A = np.dot(f.T, f)
        
    Ql = np.zeros((R*R, Qangle*Qstrength*Qcoherence, patchSize*patchSize, patchSize*patchSize))  # Eq.4
    Vl = np.zeros((R*R, Qangle*Qstrength*Qcoherence, patchSize*patchSize))                       # Eq.5
        
    Ql[t,j] += A
    im_HR_f = im_HR.ravel()[grid].reshape(patchSize*patchSize, -1)
    b1 = f.T * im_HR_f
    b = b1.reshape(-1, patchSize*patchSize)
    Vl[t,j] += b
#     mark[t,j] = mark[t,j]+1
    
    return Ql, Vl

In [6]:
# 并发方式
warnings.filterwarnings('ignore')
print('Begin to process images:')
imagecount = 1
wGaussian = Gaussian2d([patchSize, patchSize], 2)
wGaussian = wGaussian/wGaussian.max()
wGaussian = np.diag(wGaussian.ravel())
for image in filelist:
    print('\r', end='')
    print('' * 60, end='')
    print('\r Processing ' + str(imagecount) +'/' + str(len(filelist))+ ' image ('   + image + ')')
    start = time.time()
    im_uint8 = cv2.imread(image)
    if is_greyimage(im_uint8):
        im_uint8 = im_uint8[:,:,0]
    if len(im_uint8.shape)>2:
        im_ycbcr = BGR2YCbCr(im_uint8)
        im = im_ycbcr[:,:,0]
    else:
        im = im_uint8
    im = modcrop(im,R)
    H, W = im.shape
    im_LR = Prepare(im,patchSize,R)
    im_HR = im2double(im)
    #im_HR = Dog1(im_HR)                # optional: sharpen the image
    im_GX,im_GY = np.gradient(im_LR)
    index = np.array(range(im_LR.size)).reshape(im_LR.shape)
    offset = np.array(index[0:H, 0:W].ravel())
    offset2 = np.array(range(im.size))
    gridon = index[0:patchSize, 0:patchSize]
    batch = 2000
    iteration = ceil(im.size / batch + 0.000000000000001)
    i = range(iteration - 5)
    p = Pool()
    ret = p.map(ConcurenrtTrainprocessSMC, i)
    print('Con out')
    
    for i in range(iteration - 5):
        Q += np.array(ret[i][0])
        V += np.array(ret[i][1])
    end = time.time()
    print(end - start)
    
#     Q, V, mark = TrainProcessSMC(im_LR, im_HR, im_GX, im_GY,patchSize, w, Qangle, Qstrength,Qcoherence, stre, cohe, R, Q, V, mark)  # get Q, V of each patch
    imagecount += 1

Begin to process images:
 Processing 1/4 image (trainingDataSmall/153093.jpg)
Con out
29.007375955581665
 Processing 2/4 image (trainingDataSmall/169012.jpg)
Con out
48.211426734924316
 Processing 3/4 image (trainingDataSmall/176019.jpg)
Con out
44.54947304725647
 Processing 4/4 image (trainingDataSmall/181079.jpg)
Con out
39.375470876693726


In [39]:
# np.sum(kmeans.labels_==0)/kmeans.labels_.shape[0]
array = np.array(ret[5][1])
array.shape

(4, 216, 121)

In [7]:
print('Get the filter of RAISR:')
for t in range(R*R):
    print('\rType: '+str(t))
    for j in range(Qangle*Qstrength*Qcoherence):
        while(True):
            if(Q[t,j].sum()<100):
                break
            if(np.linalg.det(Q[t,j])<1):
                Q[t,j] = Q[t,j] + np.eye(patchSize*patchSize)* Q[t,j].sum()*0.000000005
            else:
                h[t,j] = np.linalg.inv(Q[t,j]).dot(V[t,j])         # Eq.2
                break

                
print('Saving...')
with open("Filters/Ckmeans_filter"+str(R), "wb") as fp:
    pickle.dump(h, fp)

with open("Filters/Ckmeans_Qfactor_str"+str(R), "wb") as sp:
    pickle.dump(stre, sp)

with open("Filters/Ckmeans_Qfactor_coh"+str(R), "wb") as cp:
    pickle.dump(cohe, cp)


print('\r', end='')
print(' ' * 60, end='')
print('\rFinished.')

Get the filter of RAISR:
Type: 0
Type: 1
Type: 2


/usr/local/anaconda3/lib/python3.7/site-packages/numpy/linalg/linalg.py:2093: RuntimeWarning: overflow encountered in det
  r = _umath_linalg.det(a, signature=signature)


Type: 3
Type: 4
Type: 5
Type: 6
Type: 7
Type: 8
Saving...
Finished.                                                   


In [8]:
print(Q)
print(h)

[[[[ 5648835.38434242  5645236.33128634  5641376.31066372 ...
     5629358.96675032  5630151.76793448  5631359.79393983]
   [ 5645236.33128634  5644013.02109747  5641364.44573461 ...
     5628141.33210211  5628947.74678882  5630149.81537777]
   [ 5641376.31066372  5641364.44573461  5640792.19197018 ...
     5627323.07909169  5628139.89117938  5629355.54729626]
   ...
   [ 5629358.96675032  5628141.33210211  5627323.07909169 ...
     5640839.31861377  5641410.05646895  5641419.83261681]
   [ 5630151.76793448  5628947.74678882  5628139.89117938 ...
     5641410.05646895  5644056.86643388  5645277.84693416]
   [ 5631359.79393983  5630149.81537777  5629355.54729626 ...
     5641419.83261681  5645277.84693416  5648873.91452312]]

  [[ 6749254.57499163  6744915.3770526   6740762.77107051 ...
     6716581.3056967   6717849.39753831  6719915.38772665]
   [ 6744915.3770526   6742929.31942837  6739987.28468809 ...
     6714546.41379353  6715804.60273629  6717847.42343604]
   [ 6740762.77107051  

In [ ]:
print(h)

In [12]:
f1 = np.array([[[1,2], [1,2]],
              [[3,4], [3,4]],
              [[1,4], [4,1]]])
print(f1.shape)
f1 = f1.reshape(2,2,3)
print(f1.T.shape)
print(np.dot(f1.T, f1))
f1 * f1

(3, 2, 2)
(3, 2, 2)
[[[[ 7 11 13]
   [15 16  4]]

  [[10 16 18]
   [22 24  6]]]


 [[[10 16 18]
   [22 24  6]]

  [[11 18 19]
   [25 28  7]]]


 [[[ 3  5  5]
   [ 7  8  2]]

  [[ 6 11  8]
   [16 20  5]]]]


array([[[ 1,  4,  1],
        [ 4,  9, 16]],

       [[ 9, 16,  1],
        [16, 16,  1]]])

# Playground

In [30]:
h[3]

array([[ 0.00324561, -0.00059579, -0.00128021, ..., -0.00578292,
         0.01777824, -0.00643502],
       [-0.01600889,  0.00809095,  0.03050988, ...,  0.00758871,
         0.01989145, -0.01325478],
       [ 0.00656672, -0.00490941, -0.00502968, ...,  0.00179324,
        -0.00832478,  0.01015856],
       ...,
       [ 0.0444698 , -0.12427632,  0.18072975, ..., -0.00603139,
         0.00686369, -0.03684298],
       [-0.0129811 , -0.00030824,  0.00968535, ...,  0.02612797,
        -0.07032155,  0.02073493],
       [-0.03408313, -0.02110304, -0.00275129, ...,  0.02475028,
        -0.04518779, -0.03648413]])

In [20]:
with open("Filters/filter"+str(R), "rb") as fp:
    h_raisr = pickle.load(fp)

In [28]:
h_raisr[0]

array([[-0.0007075 , -0.00708702, -0.01046391, ...,  0.00033279,
        -0.0004557 , -0.00411572],
       [-0.00033651, -0.00460817, -0.00756705, ..., -0.00341872,
        -0.00238595,  0.00307929],
       [-0.0040753 , -0.00427952, -0.00423952, ..., -0.00210784,
        -0.00135864, -0.00191929],
       ...,
       [ 0.05302069, -0.07342937,  0.00305579, ..., -0.17168161,
         0.15819626,  0.01565984],
       [-0.01224221, -0.0717789 ,  0.15969609, ..., -0.19601314,
         0.19350631, -0.05991437],
       [-0.15019842,  0.05535512,  0.0566023 , ...,  0.00841502,
        -0.01929808,  0.0336716 ]])

In [1]:
import scipy.io as io
import numpy as np

In [2]:
pca_matrix = io.loadmat('1000_patch_PCA_matrix.mat')['p']
#pca_matrix = np.array(pca_matrix)
pca_matrix[:,0].shape

(15,)

In [4]:
np.min(pca_matrix)

-0.20094993244945833

In [16]:
kmeans = joblib.load('2_2000_kmeans.model')

In [17]:
kmeans.predict([kmeans.cluster_centers_[2]])


array([2], dtype=int32)

In [18]:
kmeans.cluster_centers_

array([[ 1.89180212,  1.79128627],
       [-2.41173289,  1.01900477],
       [-1.18390828, -2.8171925 ],
       [ 0.1069151 , -0.08437171]])

In [7]:
mean = io.loadmat('2000_patch_mean_matrix.mat')['m']
mean.shape

(121, 1)

In [3]:
with open("Filters/Ckmeans_filter"+str(2), "rb") as fp:
    h = pickle.load(fp)

In [8]:
with open("Filters/8x_filter"+str(2), "rb") as fp:
    h = pickle.load(fp)

In [9]:
from scipy.spatial.distance import cosine

In [10]:
cosine(h[2,10].reshape(-1, 1), h[3,10].reshape(-1, 1))

0.159237958216096

In [17]:
h[0,0].shape

(121,)